# Línea base

La idea del notebook es tener una primera línea base para las estaciones de SIMAJ, el código puede funcionar como base para replicarlo a las estaciones de Mi Macro

In [1]:
import os
import sys
import osmnx as ox
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aqiGDL
%matplotlib inline

Colores y estilo de matplotlib a usar

In [7]:
plt.style.use('https://github.com/dhaitz/matplotlib-stylesheets/raw/master/pitayasmoothie-dark.mplstyle')
colors = ['7A76C2', 'ff6e9c98', 'f62196', '18c0c4', 'f3907e', '66E9EC']

Carga de estaciones de monitoreo

In [2]:
gdf_est = aqiGDL.gdf_from_db('estaciones_gdl','Estaciones')
gdf_est = ox.project_gdf(gdf_est,to_crs='EPSG:32613')
gdf_est_simaj = aqiGDL.gdf_from_db('estaciones_simaj','estaciones_simaj')
gdf_est_simaj = ox.project_gdf(gdf_est_simaj,to_crs='EPSG:32613')

,Name,x,y,geometry
0,10. Colon,-103.400806,20.606105,POINT (666650.899 2279373.835)
1,11. UVM,-103.406642,20.608323,POINT (666040.129 2279613.376)


Carga de las mediciones usando un rango de fechas

In [125]:
est = 'ATM'
inicio = '2018/12/31'
fin = '2019/04/01'
query = f"SELECT * FROM data.simaj_data_day WHERE \"FECHA\" between \'{inicio}\' and \'{fin}\'"
df = aqiGDL.df_from_query(query)
df['FECHA'] = pd.to_datetime(df['FECHA']).dt.date

Red base de Guadalajara

In [115]:
G = ox.graph_from_bbox(20.751857,20.523110,-103.201328,-103.468643)
edges = ox.graph_to_gdfs(G, nodes=False)
edges = ox.project_gdf(edges,to_crs=gdf_est_simaj.crs)

## Plot 📈

In [155]:
errors = []
for est in df.EST_SIMAJ.unique().tolist():
    try:
        fig, axes = plt.subplots(2,3,figsize=(16,9), sharex=True)
        for param, ax, color in zip(df.PARAM.unique().tolist(), axes.flatten()[1:], colors):
            df_temp = df[(df['EST_SIMAJ']==est) & (df['PARAM']==param)]
            ax.plot(df_temp['FECHA'], df_temp['CONC'], label=param, color='#'+color)
            ax.set_title(param,fontsize=15)
            if param == 'SO2' or param == 'PM10':
                ax.set_xticklabels([])
            else:
                ax.tick_params(labelrotation=45)
        
        a00 = axes[0,0]
        shax = a00.get_shared_x_axes()
        shax.remove(a00)
        edges.plot(ax=axes[0][0], color='#e8e9eb',linewidth=0.1, zorder=-1)
        edges[(edges['highway']=='primary') | (edges['highway']=='secondary')].plot(ax=axes[0][0], color='#e8e9eb',linewidth=0.5, zorder=0)
        gdf_est_simaj.plot(ax=axes[0][0], color='k', alpha=0.85, zorder=1)
        gdf_est_simaj[gdf_est_simaj['codigo']==est].plot(ax=axes[0][0], color='#ba0d38', alpha=0.85, zorder=2, markersize=90)
        axes[0][0].axis('off')
        estacion = gdf_est_simaj[gdf_est_simaj['codigo']==est]['nombre'].values[0]
        fecha_1 = df['FECHA'].min().strftime('%Y-%m-%d') 
        fecha_2 = df['FECHA'].max().strftime('%Y-%m-%d') 
        fig.suptitle(f'{estacion}\n{fecha_1} -- {fecha_2}', fontsize=20)
        plt.savefig(f'../output/figures/estsimaj/{est}_{fecha_1}.png',dpi=300)
        plt.close()
    except Exception as e:
        errors.append(est)
        pass

In [156]:
errors

[]